In [1]:
# Carga de la librerias necesarias para el entrenamiento del modelo
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sagemaker import image_uris
import pandas as pd
import numpy as np
import subprocess
import sagemaker
import warnings
import joblib
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Carga del conjunto de datos y elección de las características
filename_dataset = '20221215_151443_pre.csv'
df = pd.read_csv(filename_dataset)
X = df[['steering_angle_', 'speed', 'rpm_', 'acceleration', 'throttle_position', 'engine_temperature',
        'system_voltage', 'heart_rate', 'distance_travelled', 'latitude','longitude', 'current_weather',
        'accidents_onsite']]
y = df[['accident_rate']]
X = X.dropna(axis = 0)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [3]:
# Configurar y entrenar el modelo
mlp = MLPClassifier(hidden_layer_sizes = (10, 10, 10), max_iter = 1000)
mlp.fit(x_train, y_train.values.ravel())
predictions = mlp.predict(x_test)

In [4]:
# Exportación del modelo
with open('model.joblib', 'wb') as f:
    joblib.dump(mlp, f)

In [5]:
# Comprimiendo el Modelo
command = "tar -cvpzf model.tar.gz model.joblib Inference.py"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [6]:
# Subir el modelo a S3
s3 = boto3.session.Session().resource('s3')
session_sagemaker = sagemaker.Session()
bucket = session_sagemaker.default_bucket() 
response = s3.meta.client.upload_file('model.tar.gz', bucket, 'model.tar.gz')

In [7]:
# Obtener nombre de región y configurar el cliente
region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name = region)

In [8]:
# Rol para otorgar permiso a SageMaker para acceder a los servicios de AWS.
sagemaker_role = sagemaker.get_execution_role()

In [9]:
# Obtener modelo de S3
model_url = f"s3://{bucket}/model.tar.gz"

In [10]:
# Obtener imagen de contenedor (ejemplo prediseñado)
container = image_uris.retrieve(framework = 'sklearn', 
                                region = region, 
                                version = '0.23-1', 
                                py_version = "py3", 
                                instance_type = "ml.m5.large")

In [11]:
# Crear modelo
model_name = "accident-prediction"
model_artifacts = f"s3://{bucket}/model.tar.gz"
response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    Containers = [{
        "Image": container,
        "Mode": "SingleModel",
        "ModelDataUrl": model_url,
        "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,'SAGEMAKER_PROGRAM': 'Inference.py'} 
    }]
)

In [12]:
# Configurar el punto final
configuration_name = "endpoint-configuration-prediction-model"
response = client.create_endpoint_config(
   EndpointConfigName = configuration_name,
   ProductionVariants = [
        {
            "ModelName": model_name,
            "VariantName": "accident",
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1
        } 
    ]
)

In [13]:
# Crear el punto final
response = client.create_endpoint(
    EndpointName = "endpoint-accident-prediction",
    EndpointConfigName = configuration_name
)